## Imports


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Preparation



In [ ]:
data = pd.read_csv('../AIRPOL_data.csv', sep=';', decimal=',')

data_germany = data[(data['Country'] == 'Germany') & (data['Air_Pollutant'] == 'PM2.5')]

# Selecionar as variáveis que vão ser utilizadas:
# X1: Nível médio de poluição, X2: Área da região afetada, Y: Número de mortes prematuras
data_ger = data_germany[['NUTS_Code', 'Air_Pollution_Average[ug/m3]', 'Populated_Area[km2]', 'Value']].dropna()

# Definir as variáveis independentes (X) e dependente (Y)
X = data_ger[['Air_Pollution_Average[ug/m3]', 'Populated_Area[km2]']]
Y = data_ger['Value']

## Resolution

### a) Modelo de Regressão Linear

In [ ]:
# Adicionar constante para o modelo
X_const = sm.add_constant(X)

model = sm.OLS(Y, X_const).fit()

### b) Condições sobre os resíduos.

In [ ]:
residuos = model.resid

# Teste de normalidade dos resíduos com Shapiro-Wilk
shapiro_res = stats.shapiro(residuos)

### c) Existência de colinearidade (VIF)

In [ ]:
vif_data = pd.DataFrame()
vif_data['Variável'] = X_const.columns
vif_data['VIF'] = [variance_inflation_factor(X_const.values, i)
                   for i in range(X_const.shape[1])]

### d) Comentário sobre o modelo obtido e suas características para a sua qualidade do mesmo



In [ ]:
print('Resumo do Modelo Obtido')
print(model.summary())


print('\nTeste de normalidade dos resíduos')
print(f"Statistic: {shapiro_res.statistic:.4f}, p-value: {shapiro_res.pvalue:.4f}")
print("Se o valor de p-value > 0.05 no teste de Shapiro-Willk podemos assumir normalidade")

print('\nResiduos')
print(residuos)

print("\nValores de VIF")
print(vif_data)
print("Valores de VIF superiores a 5 ou 10 indicam colinearidade")

### e) Estimativa do número de mortes para a regiões com NUTS code

In [ ]:
nuts_codes = ['DE131', 'DE132', 'DE133', 'DE134', 'DE135', 'DE136', 'DE137', 'DE138', 'DE139']

data_ger_pred = data_ger[data_ger['NUTS_Code'].isin(nuts_codes)].copy()

if not data_ger_pred.empty:
    X_pred = data_ger_pred[['Air_Pollution_Average[ug/m3]', 'Populated_Area[km2]']]
    X_pred_const = sm.add_constant(X_pred)
    data_ger_pred['Prediction'] = model.predict(X_pred_const)
    print("\nPrevisão vs. Valores Reais:")
    print(data_ger_pred[['NUTS_Code', 'Value', 'Prediction']])
else:
    print("\nNão foram encontrados registros para os NUTS Codes especificados.")